In [17]:
# Para OCR e identificação no mapa
from pdf2image import convert_from_path
import pytesseract
import geopandas as gpd
import pandas as pd
import os
from PIL import Image
import shutil
import re

In [13]:
"""
Vai retornar o "path" absoluto de todos os ficheiros dado um diretório.
"""
def absoluteFilePaths(directory):
    paths = []
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            paths.append(os.path.abspath(os.path.join(dirpath, f)))

    return paths


"""
Lê o pdf dado e guarda como um ficheiro JPEG no diretório escolhido.
"""
def pdf2jpeg(pdf, dirname):
    pdf_name = os.path.split(pdf)[-1].split(".")[0]
    pages    = convert_from_path(pdf, 500)
    for (i, page) in enumerate(pages):
        _id = str(i)
        jpeg_path = os.path.join(dirname, pdf_name + "_" + _id + ".jpeg")
        page.save(jpeg_path, 'JPEG')

    return dirname


"""
Lê uma imagem JPEG e tranforma-a em texto utilizando a biblioteca pytesseract
"""
def jpeg2text(jpeg):
    return str(((pytesseract.image_to_string(Image.open(jpeg), lang="por"))))


"""
Esta é uma função agregadora: chama todas as funções definidas anteriormente. 
Lê uma lista de documentos PDF's e transformma-a em texto através da biblioteca Pytesseract.
Como passo temporário, ele guarda os ficheiros JPEG. 
Apenas considera os documentos pdfs com um tamanho minimo definido, ignorando os ficheiros vazios. 
Se a pasta temporária dos ficheiros JPEG já tiver documentos, ele avança o 1º passo, fazendo apenas
a conversão dos JPEG para texto, com base nas imagens existentes
"""
def pdf2text(pdfs, keep_jpegs = False, min_text_len = 300, use_existing_jpegs = None):

    if use_existing_jpegs is None:

        if keep_jpegs:
            dirname = "pytesseract_jpegs"
            if os.path.exists(dirname):
                shutil.rmtree(dirname)
            os.mkdir(dirname)
        else:
            dirname = tempfile.mkdtemp()

        print(" 1 | Populating JPEG directory ...")
        N = len(pdfs)
        for (i, pdf) in enumerate(pdfs, start = 1):
            print("%3d/%-3d" % (i, N))
            pdf2jpeg(pdf, dirname)

    elif type(use_existing_jpegs) is str:
        dirname = use_existing_jpegs

    
    jpegs = absoluteFilePaths(dirname)
    print(" 2 | Parsing JPEG using Pytesseract ...")
    N = len(jpegs)
    texts = {}
    for (i, jpeg) in enumerate(jpegs, start = 1):
        text = jpeg2text(jpeg)
        print("%3d/%-3d (Reading %12s | %5d characters)" % (i, N, os.path.split(jpeg)[-1], len(text)))
        if text is not None:
            if len(text) > min_text_len:
                nome = os.path.split(jpeg)[-1].split(".")[0]
                texts[nome] = text

    if not keep_jpegs:
            shutil.rmtree(dirname)

    return texts

# ---

texts = pdf2text(absoluteFilePaths(os.path.join("..", "resources","fichas_areas")), use_existing_jpegs = None, keep_jpegs=True)
# texts = pdf2text(absoluteFilePaths(os.path.join("..", "resources","fichas_areas")), use_existinºg_jpegs = "pytesseract_jpegs", keep_jpegs=True)

 1 | Populating JPEG directory ...
  1/6  
  2/6  
  3/6  
  4/6  
  5/6  
  6/6  
 2 | Parsing JPEG using Pytesseract ...
  1/6   (Reading     1_0.jpeg |  3928 characters)
  2/6   (Reading     2_0.jpeg |  4110 characters)
  3/6   (Reading     3_0.jpeg |  4193 characters)
  4/6   (Reading     4_0.jpeg |  3544 characters)
  5/6   (Reading     5_0.jpeg |  3871 characters)
  6/6   (Reading     6_0.jpeg |  3862 characters)


In [14]:
def primeira_letra_alpha(s):
    m = re.search(r'[a-z]', s, re.I)
    if m is not None:
        return m.start()
    return -1

In [15]:
# Definimos um DataFrame vazios e com as colunas:

# * "Ficheiro": que conterá o número das fichas
# * "Morada": que conterá a morada que se encontra nas fichas
# * "Freguesia": contém a freguesia daquelas moradas que especificam este critério
# * "Categoria da Obra": especifica que tipo de obra foi realizada: "Construção", "Ampliação", "Alteração", "Demolição"

data = pd.DataFrame(columns = ["Ficheiro", "Morada", "Freguesia", "Categoria da Obra"])

# Freguesias do município de Águeda

freguesias = ["Aguada de Cima", "Águeda e Borralha", "Barrô e Aguada de Baixo", "Belazaima do Chão, Castanheira do Vouga e Agadão",
             "Fermentelos", "Macinhata do Vouga", "Préstimo e Macieira de Alcoba", "Recardães e Espinhel", "Travassô e Óis da Ribeira",
             "Trofa, Segadães e Lamas do Vouga", "Valongo do Vouga"]



def retirar_paginas_extra(texts, ys):
    """
    Elimina textos do dicionario `texts` onde não apareçam as palavras/frases contidas na lista `ys`.
    """
    for (nome, text) in reversed(sorted(texts.items())):
        exists = False
        for y in ys:   
            if text.find(y) >= 0:
                exists = True
        
        if not exists:
            del[texts[nome]]


def procurar_categoria(text, categorias_obras):
    """
    Procura a categoria da obra, especificando, então, qual o propósito do pedido de licenciamento.

    Para isso irá procurar no texto das fichas o conteúdo da lista - categorias_obras. Guarda este conteúdo numa lista temporária. 
    Vai encontrar o valor mínimo dessa lista, de forma a encontrar a primeira categoria de obra que é mencionada na ficha de áreas. 
    Após encontrar o índice inicial desta categoria, irá ver a que categoria então é que este corresponde na lista original das categorias.
    Retorna a categoria começando com letra maiuscula. 
    """

    lista = []
    for y in categorias_obra:
        conteudo = text.find(y)
        lista.append(conteudo)

    m = min([l for l in lista if l >= 0])
    mi = lista.index(m)
    mc = categorias_obra[mi]
    return mc.capitalize()

    

def procurar_local(text, exp_init, exp_fin, query_length = 150):
    """
    Esta função está responsável por encontrar a morada na ficha de áreas. 
    Para isso pesquisa-se por duas expressões: a expressão incial que será ou Rua ou Travessa e a expressão 
    final que será uma das Freguesias listadas (lista freguesias) ou Águeda. 
    Para tal vamos definir os indices e encontrá-los na ficha de áreas, encontrando o comprimento da expressão da morada

    """
    exists = False
    indexes = []
    for exp_i in exp_init:
        index = text.find(exp_i)
        if index >= 0:
            indexes.append(index)
            exists = True
    if not exists:
        print("Can't find any of the initial expressions.")
        return np.nan
    m = max(indexes)
    content = text[m:m + query_length]

    indexes = []
    found_exp = []
    exists = False
    for exp_f in exp_fin:
        index = content.find(exp_f)
        if index >= 0:
            indexes.append(index)
            found_exp.append(exp_f)
            exists = True
    if not exists:
        print("Can't find any of the final expressions in the allocated query_length.")
        return np.nan
    M = max(indexes)
    Mi = indexes.index(M)
    exp = found_exp[Mi]
    L = len(exp)
    content = text[m:m+M+L]
    return content

ficheiro = []
morada   = []
cat      = []
locals   = {}

titulos_possiveis = ["Local da Operação Urbanística", "Local da Operação", "Local da Obra"] # Possíveis títulos
categorias_possiveis = ["Operação Urbanística", "Natureza da Obra"] # categorias da obra possíveis
categorias_obra = ["Construção", "Ampliação", "Alteração", "Demolição", "alterações"]

retirar_paginas_extra(texts, ["CARATERIZAÇÃO DA OBRA", "CARACTERIZAÇÃO DA OBRA"])

for (nome, text) in texts.items():
    ficheiro.append(nome)
    r = procurar_local(text, ["Travessa", "Rua"], freguesias + ["Águeda"])
    morada.append(r)
    x = procurar_categoria(text, categorias_obra)
    cat.append(x)

data["Ficheiro"]          = ficheiro
data["Morada"]            = morada
data["Categoria da Obra"] = cat

data



,Ficheiro,Morada,Freguesia,Categoria da Obra
0,1_0,Rua do Depósito — Rio Covo - Águeda,NaN,Construção
1,2_0,"Rua do Rio, N.º 24, Fermentelos, Águeda",NaN,Construção
2,3_0,"Rua da Alagoa, Fermentelos - Águeda",NaN,Construção
3,4_0,Rua de Moçambique — Lote 28 — Alagoa — Águeda,NaN,Ampliação
4,5_0,"Rua de Baixo, n.º 4 — Cavadas de Baixo — Valon...",NaN,Ampliação
5,6_0,"Rua Nossa Senhora da Piedade, nº 19 — Macinhat...",NaN,Alterações


In [18]:
#Retirar as expressões da seguinte lista - [N.º, n.º, nº, -, Lote, lote] presentes na morada de forma a encontrar o geocode

t = {
    "N.º": "",
    "n.º": "",
    "nº": "",
    "Lote": "",
    "lote": "",
    "—": ","
}

for i in data.index:
    data.at[i, "Morada"] = re.sub(r"N.º|n.º|nº|Lote|lote|—", lambda x: t[x.group(0)], data.loc[i]["Morada"])

In [19]:
# Atribuição manual da rua para o Ficheiro 3, devido à incompatibilidade do geocode
data.Morada[2] = "R. Alagoa, Fermentelos, Águeda"

In [20]:
#Através do geopandas definimos o encoding da morada

geometria = gpd.tools.geocode(data["Morada"])
geometria = geometria.to_crs("EPSG:3763")

In [21]:
# Filtro do documento com as coordenadas geográficas de todo o país de forma a criar um dataframe excluisivo com as freguisias do município de Águeda

freguesias_pt = gpd.read_file("../freguesias/Cont_AAD_CAOP2017.shp")
agueda = freguesias_pt[freguesias_pt["Concelho"] == "ÁGUEDA"]
agueda

DriverError: ../freguesias/Cont_AAD_CAOP2017.shp: No such file or directory

In [ ]:
# Encontrar a que freguesia corresponde cada ponto e a juntar os dois DataFrames
pontos_fichas = geometria.sjoin(agueda, how="left")
pontos_fichas

,geometry,address,index_right,Dicofre,Freguesia,Concelho,Distrito,TAA,AREA_EA_Ha,AREA_T_Ha,Des_Simpli
0,POINT (-22311.573 101148.328),"Rua do Depósito, 3750-327, Águeda, Portugal",1719,010121,União das freguesias de Águeda e Borralha,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,3602.93,3602.93,Águeda e Borralha
1,POINT (-33953.301 99855.000),"Rua do Rio, 3750-432, Águeda, Portugal",1687,010109,Fermentelos,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,858.20,858.20,Fermentelos
2,POINT (-33359.970 100326.122),"Rua Alagoa, 3750-455, Águeda, Portugal",1687,010109,Fermentelos,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,858.20,858.20,Fermentelos
3,POINT (-27188.059 102325.326),"Rua de Moçambique, 3750-301, Águeda, Portugal",1719,010121,União das freguesias de Águeda e Borralha,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,3602.93,3602.93,Águeda e Borralha
4,POINT (-26178.024 106910.359),"Rua da Cavada, 3750-803, Águeda, Portugal",232,010119,Valongo do Vouga,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,4320.11,4320.11,Valongo do Vouga
5,POINT (-26656.919 109625.289),"Rua Nossa Senhora da Piedade, Águeda, Portugal",1766,010112,Macinhata do Vouga,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,3195.44,3195.44,Macinhata do Vouga


In [ ]:
# Juntar o DataFrame anterior com o inicial do data (com as informações da morada, da categoria de construção)

fichas = pd.merge(pontos_fichas, data, left_index=True, right_index=True)
fichas = fichas.drop(columns=["Morada", "Freguesia_y"])

# Reordenar as colunas do DF

fichas = fichas [["Ficheiro", "geometry", "address", "index_right", "Dicofre", "Freguesia_x", "Concelho", 
"Distrito","TAA", "AREA_EA_Ha", "AREA_T_Ha", "Des_Simpli", "Categoria da Obra"]]

In [ ]:
fichas

,Ficheiro,geometry,address,index_right,Dicofre,Freguesia_x,Concelho,Distrito,TAA,AREA_EA_Ha,AREA_T_Ha,Des_Simpli,Categoria da Obra
0,1_0,POINT (-22311.573 101148.328),"Rua do Depósito, 3750-327, Águeda, Portugal",1719,010121,União das freguesias de Águeda e Borralha,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,3602.93,3602.93,Águeda e Borralha,Construção
1,2_0,POINT (-33953.301 99855.000),"Rua do Rio, 3750-432, Águeda, Portugal",1687,010109,Fermentelos,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,858.20,858.20,Fermentelos,Construção
2,3_0,POINT (-33359.970 100326.122),"Rua Alagoa, 3750-455, Águeda, Portugal",1687,010109,Fermentelos,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,858.20,858.20,Fermentelos,Construção
3,4_0,POINT (-27188.059 102325.326),"Rua de Moçambique, 3750-301, Águeda, Portugal",1719,010121,União das freguesias de Águeda e Borralha,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,3602.93,3602.93,Águeda e Borralha,Ampliação
4,5_0,POINT (-26178.024 106910.359),"Rua da Cavada, 3750-803, Águeda, Portugal",232,010119,Valongo do Vouga,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,4320.11,4320.11,Valongo do Vouga,Ampliação
5,6_0,POINT (-26656.919 109625.289),"Rua Nossa Senhora da Piedade, Águeda, Portugal",1766,010112,Macinhata do Vouga,ÁGUEDA,AVEIRO,ÁREA PRINCIPAL,3195.44,3195.44,Macinhata do Vouga,Alterações


In [ ]:
type(fichas)

geopandas.geodataframe.GeoDataFrame

In [ ]:
fichas.to_csv("dados_ocr_fichas.csv") 